# CUPS Report Analisis Function

In [10]:
import pandas as pd
import numpy as np
list = range(100)
np.mean(list)

np.float64(49.5)

In [23]:
df = pd.read_excel('/home/raul/CAREER/DATA_SCIENCE/proyectos/CUPX/cups-examples/InformeCUPS_AMOROS.xlsx', sheet_name=1)
df.drop('Tipo lectura', axis=1, inplace=True)
df

,Fecha lectura anterior,Fecha lectura,Consumo P1,Consumo P2,Consumo P3,Consumo P4,Consumo P5,Consumo P6
0,30/11/2020,01/12/2020,0,1122,771,0,0,1455
1,01/12/2020,01/01/2021,33773,23115,0,0,0,50226
2,01/01/2021,01/02/2021,36772,24989,0,0,0,44394
3,01/02/2021,01/03/2021,32760,22241,0,0,0,40610
4,01/03/2021,01/04/2021,0,33989,23429,0,0,50449
5,01/04/2021,01/05/2021,0,0,0,36068,25461,49314
6,01/05/2021,31/05/2021,0,0,0,33233,24018,47816
7,31/05/2021,30/06/2021,0,0,30461,22779,0,37340
8,30/06/2021,31/07/2021,32269,23738,0,0,0,44815
9,31/07/2021,31/08/2021,0,0,37129,26828,0,52685


In [25]:
df.columns

Index(['Fecha lectura anterior', 'Fecha lectura', 'Consumo P1', 'Consumo P2',
       'Consumo P3', 'Consumo P4', 'Consumo P5', 'Consumo P6'],
      dtype='object')

In [48]:
class CupsARP:

    def __init__(self, path):
        self.path = path
        self.page_list = []

        self.df_cups_list = pd.read_excel(path, sheet_name=0)
        self.df_activa = pd.read_excel(path, sheet_name=1).drop('Tipo lectura', axis=1)
        self.df_maxim = pd.read_excel(path, sheet_name=2)
        self.df_react = pd.read_excel(path, sheet_name=3)

        self.page_list.extend([self.df_cups_list, self.df_activa, self.df_maxim, self.df_react])

    
    def load_page(self, param):
        return self.page_list[param]


    def ly_consum(self):
        self.consums = {}
        
        self.tot_cons = self.df_activa[['Fecha lectura','Consumo P1', 'Consumo P2',
       'Consumo P3', 'Consumo P4', 'Consumo P5', 'Consumo P6']].sum().sum()
        self.consums['TOTAL'] = self.tot_cons

        for i in range(1,7):
             name = f'P{i}'
             
             cons = self.df_activa.loc[:,f'Consumo P{i}'].sum()
             self.consums[name] = cons

        return self.consums
    
    def year_table(self):
        # Crear una tabla pivotada con los meses y periodos P1 a P6
        tabla = self.df_activa[['Fecha lectura', 'Consumo P1', 'Consumo P2',
                                'Consumo P3', 'Consumo P4', 'Consumo P5', 'Consumo P6']].copy()

        # Convertir las fechas a meses (ej. Enero, Febrero, ...)
        tabla['Mes'] = pd.to_datetime(self.df_activa['Fecha lectura'], format="%d/%m/%Y").dt.strftime('%B').str.upper()

        # Agrupar los datos por 'Mes' y sumar los consumos para cada mes y cada periodo
        tabla_agrupada = tabla.groupby('Mes').sum()

        # Calcular el total por cada mes
        tabla_agrupada['Total mes'] = tabla_agrupada.sum(axis=1)

        # Calcular el total por cada periodo (columna)
        total_periodos = tabla_agrupada[['Consumo P1', 'Consumo P2', 'Consumo P3',
                                         'Consumo P4', 'Consumo P5', 'Consumo P6']].sum()

        # Añadir una fila de total de periodos al final de la tabla
        tabla_agrupada.loc['Total periodo'] = total_periodos
        tabla_agrupada.loc['Total periodo', 'Total mes'] = total_periodos.sum()

        return tabla_agrupada



In [49]:
cups = CupsARP('/home/raul/CAREER/DATA_SCIENCE/proyectos/CUPX/cups-examples/InformeCUPS_AMOROS.xlsx')
print(cups.consum())

{'TOTAL': np.int64(3312940), 'P1': np.int64(327065), 'P2': np.int64(373459), 'P3': np.int64(395411), 'P4': np.int64(471676), 'P5': np.int64(192998), 'P6': np.int64(1552331)}


In [50]:
activa = cups.load_page(1)

In [53]:
tabla =activa[['Fecha lectura', 'Consumo P1', 'Consumo P2',
                                'Consumo P3', 'Consumo P4', 'Consumo P5', 'Consumo P6']].copy()
tabla

,Fecha lectura,Consumo P1,Consumo P2,Consumo P3,Consumo P4,Consumo P5,Consumo P6
0,01/12/2020,0,1122,771,0,0,1455
1,01/01/2021,33773,23115,0,0,0,50226
2,01/02/2021,36772,24989,0,0,0,44394
3,01/03/2021,32760,22241,0,0,0,40610
4,01/04/2021,0,33989,23429,0,0,50449
5,01/05/2021,0,0,0,36068,25461,49314
6,31/05/2021,0,0,0,33233,24018,47816
7,30/06/2021,0,0,30461,22779,0,37340
8,31/07/2021,32269,23738,0,0,0,44815
9,31/08/2021,0,0,37129,26828,0,52685


In [55]:
tabla['Mes'] = pd.to_datetime(activa['Fecha lectura'], format="%d/%m/%Y").dt.strftime('%B').str.upper()
tabla

,Fecha lectura,Consumo P1,Consumo P2,Consumo P3,Consumo P4,Consumo P5,Consumo P6,Mes
0,01/12/2020,0,1122,771,0,0,1455,DECEMBER
1,01/01/2021,33773,23115,0,0,0,50226,JANUARY
2,01/02/2021,36772,24989,0,0,0,44394,FEBRUARY
3,01/03/2021,32760,22241,0,0,0,40610,MARCH
4,01/04/2021,0,33989,23429,0,0,50449,APRIL
5,01/05/2021,0,0,0,36068,25461,49314,MAY
6,31/05/2021,0,0,0,33233,24018,47816,MAY
7,30/06/2021,0,0,30461,22779,0,37340,JUNE
8,31/07/2021,32269,23738,0,0,0,44815,JULY
9,31/08/2021,0,0,37129,26828,0,52685,AUGUST


In [ ]:
tabla.groupby('Mes')